<table border="0" align="left" width="700" height="144">
<tbody>
<tr>
<td width="120"><img width="100" src="https://static1.squarespace.com/static/5992c2c7a803bb8283297efe/t/59c803110abd04d34ca9a1f0/1530629279239/" /></td>
<td style="width: 600px; height: 67px;">
<h1 style="text-align: left;">__Dunderscores__</h1>
<p><a href="https://colab.research.google.com/github/KenzieAcademy/python-notebooks/blob/master/demo_underscores.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left" width="188" height="32" /> </a></p>
</td>
</tr>
</tbody>
</table>

Excerpted from “Python Tricks: The Book.” by Dan Bader and also from [The Meaning of Underscores in Python](https://dbader.org/blog/meaning-of-underscores-in-python).

Single and double underscores can have a certain meaning in Python. Some of that meaning is merely by convention and intended as a hint to the programmer, and some of it is actually enforced by the Python interpreter.

There are five patterns and naming conventions, and they **do** affect the behavior of your Python programs.
 - Single leading underscore: `_var`
 - Single trailing Underscore: `var_`
 - Double leading underscore: `__var`
 - Double leading and trailing underscore: `__var__`
 - Single underscore: `_`

It is useful to know these conventions when you are reviewing others' code.

## Single Leading Underscore: `_var`

The single underscore prefix means that a variable or function is intended for internal use. This convention is defined in [PEP 8](https://www.python.org/dev/peps/pep-0008/#descriptive-naming-styles).

This convention is **not** enforced by the Python interpreter, for example, in the way that public and private variables are declared and enforced in some other languages. It is a hint from the programmer that the item is not really meant for public use.


In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23

print(Test().foo)
print(Test()._bar)

As you can see, the leading single underscore in `_bar` did not prevent us from “reaching into” the class and accessing the value of that variable.

The single underscore prefix in Python is merely an agreed-upon convention. But, what happens if we try to import an underscore function from a module?

In [ ]:
# paste this into a my_module.py
# def external_func():
#     return 23
# def _internal_func():
#     return 42

from my_module import *
print(external_func())
print(_internal_func())

Wildcard imports (e.g., `from module_name import *`) should be avoided, as they make it unclear which names are present in the namespace. It's better to stick to regular imports for the sake of clarity. Unlike wildcard imports, regular imports are not affected by the leading single underscore naming convention.


In [ ]:
# replace the last few lines of my_module.py with these
import my_module
print(my_module.external_func())
print(my_module._internal_func())

##  Single Trailing Underscore: `var_`

Sometimes the most fitting name for a variable is already taken by a keyword in the Python language. Therefore, names like `class`, `def`, or `list` should not be used as variable names in Python. If you just cannot resist using them, you can append a single underscore to break the naming conflict.

In [ ]:
def make_object(name, list_):
    pass

In [ ]:
list_ = [n for n in range(10)]

## Double Leading Underscore: `__var`

The naming patterns we've covered so far receive their meaning from agreed-upon conventions only. With Python class attributes (variables and methods belonging to a class) that start with double underscores, things are a little different.

A double underscore prefix causes the Python interpreter to rewrite the attribute name in order to avoid naming conflicts in subclasses. This is called _name mangling_. The interpreter changes the name of the variable in a way that makes it harder to create collisions when the class is extended later.

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 42
    
    def go_baz(self):
      print(self.__baz)

In [ ]:
t = Test()
dir(t)

This gives us a list with the object’s attributes. Let's take this list and look for our original variable names `foo`, `_bar`, and `__baz`. 

 - `self.foo` variable appears unmodified as `foo` in the attribute list.
 - `self._bar` behaves the same way &mdash; it shows up on the class as `_bar`.

### What happened to poor __baz?

There's an attribute called `_Test__baz` on this object. This is the name mangling that the Python interpreter applies. It does this to protect the variable from being overridden in subclasses.

In [ ]:
# Let's extend the Test class and override the attributes
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

In [ ]:
# Let's take a look at the overridden attributes (expecting an error here)
et = ExtendedTest()
print(et.foo)
print(et._bar)
print(et.__baz)

In [ ]:
# Check out the mangled names!
print(dir(et))

In [ ]:
class ManglingTest:
    def __init__(self):
        self.__mangled = "help I've been mangled!"

    def get_mangled(self):
        return self.__mangled

mt = ManglingTest()
print(mt.get_mangled())
print(mt.__mangled)

Name mangling affects all names that start with two underscore characters (“dunders”) in a class context &mdash; even method names.

In [ ]:
class MangledMethod:
    def __method(self):
        return "Hello from mmmmangled __method"

    def call_it(self):
        return self.__method()

# Try to invoke it from the outside, with normal dotted notation
MangledMethod().__method()

In [ ]:
# Only works from inside the class itself!
MangledMethod().call_it()

#### A Bit of Subversion


In [ ]:
_MangledGlobal__mangled = 23

class MangledGlobal:
    def test(self):
        # notice that we never even define this in class scope
        return __mangled

mg = MangledGlobal()
print(mg.test())

This demonstrated that name mangling isn’t tied to class attributes specifically. It applies to any name starting with two underscore characters used in a class context.

## Double Leading and Trailing Underscore: `__var__`
Name mangling is **not** applied if a name starts and ends with double underscores. Variables surrounded by a double underscore prefix and postfix are left unscathed by the Python interpeter.

*Rule of thumb*: Don't use this naming method for your own variables. It's reserved for special use within the Python language itself.

In [ ]:
class LeadingTrailingDunder:
    def __init__(self):
        self.__my_own_dunder__ = 1234

LeadingTrailingDunder().__my_own_dunder__

Much of Python's syntax is actually enabled by these types of "dunder" methods, often called "special" methods or "magic" methods since they seem to work like magic.
* `==` is enabled by `__eq__()`
* `>` is enabled by `__gt__()`
* `[]` is enabled by `__getitem__()`, `__setitem__()`
* ...and much more.

In [ ]:
print("1 == 1? ", int.__eq__(1, 1))  # equivalent to 1 == 1
print("1 == 2? ", int.__eq__(1, 2))  # equivalent to 1 == 2

In [ ]:
print("1 > 0? ", int.__gt__(1, 0))  # equivalent to 1 > 0
print("1 > 2? ", int.__gt__(1, 2))  # equivalent to 1 > 2

In [ ]:
L = ['a', 'z', 'c']
print(L.__getitem__(2))  # equivalent to L[2]

L.__setitem__(1, 'b')  # equivalent to L[1] = 'b'
print(L)

Even Python's entire iteration protocol used with looping is based on these "dunder" methods!

In [ ]:
# re-creating the "for" loop
L = [1, 2, 3]

def time_to_get_loopy(items):
    it = iter(items)  # turn the iterable into an iterator object
    while True:
        try:
            print(it.__next__())  # get the next value from the iterator
        except StopIteration:
            # no more items to get -- a friendly exception!
            print("End of loop.")
            break

time_to_get_loopy(L)

### Fun Detour! &mdash; Remember decorators?

In [ ]:
# how about applying the iterator protocol as a loop decorator?
L = [3, 4, 5]

def time_to_get_loopy(items):
    def outer(func):
        it = iter(items)
        def inner():
            while True:
                try:
                    func(it.__next__())
                except StopIteration:
                    # no more items to get -- a friendly exception
                    break
        return inner  # returns the newly-decorated function
    return outer  # returns a function that can act as a decorator


@time_to_get_loopy(L)  # decorator with args!
def all_the_things(item):
    print(item)


# these two lines are equivalent to applying the decorator above the function
# time_to_get_loopy = time_to_get_loopy(L)
# all_the_things = time_to_get_loopy(all_the_things)

all_the_things()

## Single Underscore: `_`
Per convention, a single standalone underscore is sometimes used as a name to indicate that a variable is temporary or insignificant.

For example, in the following loop we don't need access to the running index and we can use `_` to indicate that it is just a temporary value.

In [ ]:
for _ in range(5):
    print("waiting ...")

You can also use single underscores in unpacking expressions as a “don't care” variable to ignore particular values.

The following code example unpacks a car tuple into separate variables, but we're only interested in the values for color and mileage. However, in order for the unpacking expression to succeed we need to assign all values contained in the tuple to variables. That's where `_` is useful as a placeholder variable.

In [ ]:
car = ('red', 'auto', 2019, 3812.4)
color, _, _, mileage = car

print(color)
print(mileage)

It's also useful in a similar way for extracting particular pieces of data from a sequence all at once. By attaching the `*` operator to a variable name on the left side of an assignment operation, you can indicate that a variable should encompass anything that has not otherwise been assigned.

In [ ]:
race_times = (3.2, 4.5, 5.2, 5.9)  # try adding longer race times -- no need to change the following line
fastest, *_, slowest = race_times

print(f"The fastest time is {fastest} seconds.")
print(f"The slowest time is {slowest} seconds.")

Again, this meaning is “per convention” only and there's no special behavior triggered in the Python interpreter. The single underscore is simply a valid variable name that's sometimes used for this purpose.

#### Single Underscores in Numbers (3.6+)
A single underscore can also be used to make long numbers more readable. Python ignores underscores in a number when storing it. Even if the digits are not grouped in threes, the value will still be unaffected. When a number containing underscores is printed, only the digits are displayed. This works for both ints and floats.

In [ ]:
long_number = 12_000_000  # twelve million
print(long_number)

misplaced = 12_0000_00  # twelve million
print(misplaced)

long_decimal = 12_000_000.03
print(long_decimal)

## Conclusion
#### A handy cheat sheet

|Pattern  |Example  |Meaning |   |   |
|---|---|---|---|---|
|Single Leading Underscore   | `_var`  | Naming convention indicating a name is meant for internal use. Generally not enforced by the Python interpreter (except in wildcard imports) and meant as a hint to the programmer only.  |   |   |
|Single Trailing Underscore   |`var_`   |Used by convention to avoid naming conflicts with Python keywords.  |   |   |
|Double Leading Underscore   |`__var`   | Triggers name mangling when used in a class context. Enforced by the Python interpreter.  |   |   |
|Double Leading and Trailing Underscore  |`__var__`   |Indicates special methods defined by the Python language. Avoid this naming scheme for your own attributes.   |   |   |
|Single Underscore   |`_`   |Sometimes used as a name for temporary or insignificant "don't care" variables. Also: A digit separator for making long numbers in code easier to read.   |   |   |
		